In [1]:
import pandas as pd
import numpy as np

In [94]:
wages = pd.read_csv("wage data")
marijuana = pd.read_csv("marijuana years df")
marijuana = marijuana.iloc[0:51]
marijuana = marijuana.set_index("State")
marijuana = marijuana[["Year legalized (Rec)", "Year legalized (Med)"]]
marijuana = marijuana.rename(columns={"Year legalized (Rec)" : "rec", "Year legalized (Med)" : "med"})



In [95]:
# Create a list of all possible state-year combinations from 2001 to 2022
state_list = marijuana.index.unique()
year_range = list(map(str, range(2001, 2023)))  # Convert years to strings
index = pd.MultiIndex.from_product([state_list, year_range], names=["State", "Year"])

# Create an empty DataFrame with the multi-index
pivoted_df = pd.DataFrame(index=index)



In [96]:
pivoted_df.insert(loc=0,column="rec_legal",value=np.nan)
pivoted_df.insert(loc=1,column="med_legal",value=np.nan)

In [106]:
#would be more efficient to this the other way around, but I realzied that after starting and on a 10^4 dataset running once I can't be bothered to rewrite
for state, year in pivoted_df.index:
    for manner in ["rec", "med"]:
        value = marijuana[manner].loc[state]
        if isinstance(value, (float, int)) and not pd.isnull(value) and value <= int(year):
            pivoted_df.loc[(state, year), f"{manner}_legal"] = True
        else: 
            pivoted_df.loc[(state, year), f"{manner}_legal"] = False


In [107]:
pivoted_df.head(25)

rec_legal med_legal
State      Year                    
California 2001     False      True
           2002     False      True
           2003     False      True
           2004     False      True
           2005     False      True
           2006     False      True
           2007     False      True
           2008     False      True
           2009     False      True
           2010     False      True
           2011     False      True
           2012     False      True
           2013     False      True
           2014     False      True
           2015     False      True
           2016      True      True
           2017      True      True
           2018      True      True
           2019      True      True
           2020      True      True
           2021      True      True
           2022      True      True
Alaska     2001     False      True
           2002     False      True
           2003     False      True